In [ ]:
!pip install -U ginza ja-ginza

# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

# 作業フォルダへの移動を行います。
# もしアップロードした場所が異なる場合は作業場所を変更してください。
import os
os.chdir('/content/drive/MyDrive/ai_app_dev/4章') #ここを変更

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 7.3 MB/s eta 0:00:00
Mounted at /content/drive


## 形態素解析をやってみよう

In [ ]:
import spacy

text = '私は6月17日に友達の家に行ってから温泉に行く予定だ'

nlp = spacy.load('ja_ginza')

doc = nlp(text)
for token in doc:
  print(token)

私
は
6
月
17
日
に
友達
の
家
に
行っ
て
から
温泉
に
行く
予定
だ


In [ ]:
for token in doc:
  print(token.text, token.lemma_, token.pos_, token.tag_)

私 私 PRON 代名詞
は は ADP 助詞-係助詞
6 6 NUM 名詞-数詞
月 月 NOUN 名詞-普通名詞-助数詞可能
17 17 NUM 名詞-数詞
日 日 NOUN 接尾辞-名詞的-助数詞
に に ADP 助詞-格助詞
友達 友達 NOUN 名詞-普通名詞-一般
の の ADP 助詞-格助詞
家 家 NOUN 名詞-普通名詞-一般
に に ADP 助詞-格助詞
行っ 行く VERB 動詞-非自立可能
て て SCONJ 助詞-接続助詞
から から ADP 助詞-格助詞
温泉 温泉 NOUN 名詞-普通名詞-一般
に に ADP 助詞-格助詞
行く 行く VERB 動詞-非自立可能
予定 予定 NOUN 名詞-普通名詞-サ変可能
だ だ AUX 助動詞


## 係り受け/固有表現抽出をやってみよう

In [ ]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True, options={'distance':60})

In [ ]:
import ginza
for span in ginza.bunsetu_spans(doc):
  for token in span.lefts:
    print(ginza.bunsetu_span(token), span)

友達の 家に
6月17日に 行ってから
家に 行ってから
行ってから 行く
温泉に 行く
私は 予定だ
行く 予定だ


In [ ]:
text = '私は6月17日に静岡にある山田の家に行く予定だ'
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
for token in doc:
  print(token.text, token.ent_type_)

私 
は 
6 Date
月 Date
17 Date
日 Date
に 
静岡 Province
に 
ある 
山田 Person
の 
家 
に 
行く 
予定 
だ 


## 形態素解析をつかいこなそう

In [ ]:
input_text = '私は6月17日に静岡にある山田の家に行く予定だ'

pos_dic = {'名詞':'NOUN', '代名詞':'PRON',
           '固有名詞':'PROPN','動詞':'VERB'}
select_pos = ['名詞']

doc = nlp(input_text)
output_word = []
tg_pos = [pos_dic[x] for x in select_pos]
for token in doc:
  if token.pos_ in tg_pos:
    output_word.append(token.lemma_)
print(output_word)

['月', '日', '家', '予定']


In [ ]:
tg_pos

['NOUN']

In [ ]:
import pandas as pd

data = pd.read_csv('data/survey.csv')
print(len(data))
data.head()

86


,datetime,comment,satisfaction
0,2019/3/11,駅前に若者が集まっている(AA駅),1
1,2019/2/25,スポーツできる場所があるのが良い,5
2,2019/2/18,子育て支援が嬉しい,5
3,2019/4/9,保育園に入れる（待機児童なし）,4
4,2019/1/6,駅前商店街が寂しい,2


In [ ]:
data = data.dropna()
print(len(data))

84


In [ ]:
input_text = data['comment']
print(input_text)

0               駅前に若者が集まっている(AA駅)
1                スポーツできる場所があるのが良い
2                       子育て支援が嬉しい
3                 保育園に入れる（待機児童なし）
4                       駅前商店街が寂しい
                 ...             
81              小学校の校庭が芝生なのでとても良い
82    ホームページからアンケートを投稿できるようにしてほしい
83                  公園に遊び道具が少なすぎる
84                  もっと公園を増やしてほしい
85                  駅前に駐車場が少ない、不便
Name: comment, Length: 84, dtype: object


In [ ]:
input_text = ' '.join(input_text)
print(input_text)

駅前に若者が集まっている(AA駅) スポーツできる場所があるのが良い 子育て支援が嬉しい 保育園に入れる（待機児童なし） 駅前商店街が寂しい 生活は便利だけど遊ぶ場所がない 遊ぶ場所がない 商業施設が出来て欲しい 病院が充実している サイクリングコースが良い お祭りをもっと盛り上げて欲しい 小学校が綺麗で嬉しい 公園がもっと欲しい 近くに公園があって住みやすい スーパーまで行くのが大変 子育てしやすい デートスポットが欲しい もっと商店街を盛り上げてほしい 夜道が暗い(BB地区) ゴミ処理が楽 映画館が欲しい もっと街路樹が欲しい もう少し公園を追加して欲しい 観光スポットがない 子育てがしやすい 道が綺麗 都内へのアクセスが良い 家賃がリーズナブル BBBの渋滞をなんとかして欲しい 商店街に屋根がないから雨が降ると大変 商店街にもっとたくさん喫茶店が欲しい スポーツジムがあってよく利用している スポーツジムの利用料金が高い 働くママの支援をもっと増やして欲しい バスがあまりこない 自転車で走りやすい ランニングとか運動できる場所が多い 有名な企業が多い 駅前に駐車場が足りない 変なおじさんに声を掛けられた事がある。警察の巡回をもっと強化してほしい 消防団が活発で安心できる 災害時の避難場所が少ない気がする。 最近川の氾濫被害が大きく取り扱われているが、この町ではどのような氾濫防止を取っているか説明してほしい 信号のない交差点がｘｘｘにあり事故が起きそうで怖い、信号を付けて欲しい。 商店街の活性化活動をもっと行政が後押ししてほしい 子どもが安全に遊ぶ場所がない 駅前に駐輪場が無いので不便 街路樹の落ち葉の掃除が大変、街路樹をやめてほしい 災害時の備蓄状況はどうなっているか 歩道が広い道が多くて安心できる 地域の自治体をもっと支援してほしい 自治体の活動が頻繁で安心して暮らせる 観光地が少ない 最近他県ナンバーの車が多い、防犯はちゃんと出来ているのか 駐車場の数が少ないし、料金が高い。駅前に公共駐車場を作って欲しい 歩行者用信号が変わるのが早い。老人や子供の事を考えて設定してほしい 急行が止まるので都内へ出やすくて良い バスの路線を増やしてほしい 春の桜並木が最高に綺麗。もっと桜並木を増やしてほしい^^ 隣町にできたごみ処理施設が心配、公害は大丈夫？ 市長が若くて活気

In [ ]:
pos_dic = {'名詞':'NOUN', '代名詞':'PRON',
           '固有名詞':'PROPN','動詞':'VERB'}
select_pos = ['名詞']

doc = nlp(input_text)
output_word = []
tg_pos = [pos_dic[x] for x in select_pos]
for token in doc:
  if token.pos_ in tg_pos:
    output_word.append(token.lemma_)
print(output_word)

['駅前', '若者', 'AA', '駅', '場所', '子育て', '支援', '保育園', '待機児童', 'なし', '駅前', '商店街', '生活', '場所', '場所', '商業', '施設', '病院', 'サイクリングコース', 'お祭り', '小学校', '公園', '近く', '公園', 'スーパー', '子育て', 'デート', 'スポット', '商店街', '夜道', 'BB', '地区', 'ゴミ', '処理', '映画館', '街路樹', '公園', '観光', 'スポット', '子育て', '道', '都内', 'アクセス', '家賃', 'リーズナブル', 'bbb', '渋滞', '商店街', '屋根', '雨', '商店街', '喫茶店', '利用料金', 'ママ', '支援', 'バス', '自転車', 'ランニング', '場所', '企業', '駅前', '駐車場', 'おじ', 'さん', '声', '事', '警察', '巡回', '消防団', '災害', '時', '避難', '場所', '気', '川', '氾濫', '被害', '町', '氾濫', '防止', '信号', '交差点', 'xxx', '事故', '信号', '商店街', '活性化', '活動', '行政', '子ども', '場所', '駅前', '駐輪場', '不便', '街路樹', '落ち葉', '掃除', '街路樹', '災害', '時', '備蓄', '状況', '歩道', '道', '地域', '自治体', '自治体', '活動', '観光地', '他県', 'ナンバー', '車', '防犯', '駐車場', '数', '料金', '駅前', '公共', '駐車場', '歩行者', '用', '信号', '老人', '子供', '事', '急行', '都内', 'バス', '路線', '春', '桜', '並木', '桜並木', '隣町', 'ごみ', '処理', '施設', '心配', '公害', '市長', '活気', 'アンケート', '街づくり', '姿勢', '先月', '職員', '不正', '事件', '役所', '担当者', '気', '役所', '出張所', '駅前', '役所', '災害', '時', '避難', '

In [ ]:
output_df = pd.DataFrame({'Word':output_word})
display(output_df.head())

output_df = output_df.groupby('Word',as_index=False).size()
display(output_df.head())

,Word
0,駅前
1,若者
2,AA
3,駅
4,場所


,Word,size
0,AA,1
1,BB,1
2,bbb,1
3,xxx,1
4,おじ,1


In [ ]:
output_df.sort_values('size',ascending=False,inplace=True)
display(output_df.head())

output_df.set_index('Word', inplace=True)
display(output_df.head())

,Word,size
135,駅前,7
54,場所,6
41,公園,5
49,商店街,5
71,役所,5


,size
Word,
駅前,7
場所,6
公園,5
商店街,5
役所,5
